In [26]:
import os
from checker import *
from entities import *
from utils import *

# !mvn clean package

Análisis de instancias de datasets "a", "b" y "x"

In [27]:
datasets = {"a": {}, "b": {}, "x": {}}
instances = {}

In [28]:
%%capture

for dataset in ["a", "b", "x"]:
    for file in os.listdir(f"datasets/{dataset}"):
        instanceId = file.split("_")[1].split(".")[0]
        instance = Instance(dataset, instanceId, f"datasets/{dataset}/{file}")
        instances[f"{dataset}/{instanceId}"] = instance
        datasets[dataset][instanceId] = instance

In [30]:
display_stats_table(datasets["a"].values())

,instance,#aisles,#orders,#items,LB,UB,mean_aisle_capacity,mean_order_size,mean_items_per_aisle,mean_items_per_order
0,a/0001,116,61,155,30,68,62.78,3.70,4.07,2.61
1,a/0002,33,7,7,0,2,19.91,1.57,1.09,1.00
2,a/0003,124,82,246,33,106,84.55,4.20,6.32,3.04
3,a/0004,91,16,59,4,40,29.14,4.44,1.68,3.69
4,a/0005,161,2625,6407,1322,4395,1365.15,5.26,109.44,3.49
5,a/0006,184,10341,7089,452,3892,893.36,1.42,121.28,1.00
6,a/0007,180,8320,5747,1306,3847,872.04,1.31,109.03,1.00
7,a/0008,168,2185,5831,1304,2840,1282.26,5.97,103.04,3.75
8,a/0009,304,70,222,52,153,35.58,3.99,2.86,3.31
9,a/0010,383,1602,3689,416,1746,147.66,2.90,33.23,2.46


In [31]:

display_stats_table(datasets["b"].values())

,instance,#aisles,#orders,#items,LB,UB,mean_aisle_capacity,mean_order_size,mean_items_per_aisle,mean_items_per_order
0,b/0001,185,11321,7675,1206,3968,985.13,1.32,140.59,1.00
1,b/0002,165,2162,5922,626,3405,1139.02,6.64,99.51,3.92
2,b/0003,164,2070,5617,585,2150,1126.27,6.29,94.54,3.84
3,b/0004,163,2448,6271,701,2802,1319.90,5.80,107.75,3.65
4,b/0005,179,6468,4959,996,2869,705.64,1.47,88.96,1.00
5,b/0006,165,1857,4982,496,3558,1072.66,5.05,89.86,3.47
6,b/0007,165,1841,1621,91,833,295.44,1.27,36.46,1.00
7,b/0008,398,12334,10365,2235,6120,313.38,1.21,93.09,1.00
8,b/0009,417,5581,4990,1123,3205,118.60,1.31,35.72,1.00
9,b/0010,482,14952,13510,1228,9361,722.49,1.26,188.81,1.00


In [32]:
display_stats_table(datasets["x"].values())

,instance,#aisles,#orders,#items,LB,UB,mean_aisle_capacity,mean_order_size,mean_items_per_aisle,mean_items_per_order
0,x/0001,164,1949,4608,810,2380,770.74,3.78,79.86,2.83
1,x/0002,168,2942,2485,474,1613,515.86,1.36,55.10,1.00
2,x/0003,180,11330,7399,1831,3958,1032.26,1.32,132.56,1.00
3,x/0004,172,2732,6544,354,4289,1324.35,5.30,105.12,3.66
4,x/0005,182,11024,7365,1628,5099,961.25,1.35,130.27,1.00
5,x/0006,168,2704,6523,1196,2979,1353.11,5.22,110.38,3.62
6,x/0007,418,8725,7715,1490,5518,162.80,1.27,52.93,1.00
7,x/0008,482,20364,17991,3937,10520,926.88,1.22,243.01,1.00
8,x/0009,483,39701,33994,6154,19163,1447.50,1.26,395.89,1.00
9,x/0010,482,44193,36991,7668,14433,1606.03,1.24,437.42,1.00


Selección de instancias para experimentos

In [33]:
experiment_instances = {
    "a": [i for i in datasets["a"].values() if i.id in ["0001", "0004", "0009", "0017", "0019"]],
    "b": [i for i in datasets["b"].values() if i.id in ["0001", "0003", "0005", "0007", "0009"]],
    "x": [i for i in datasets["x"].values() if i.id in ["0001", "0003", "0006", "0007", "0008"]]
}

In [34]:
display_stats_table(experiment_instances["a"])

,instance,#aisles,#orders,#items,LB,UB,mean_aisle_capacity,mean_order_size,mean_items_per_aisle,mean_items_per_order
0,a/0001,116,61,155,30,68,62.78,3.70,4.07,2.61
1,a/0004,91,16,59,4,40,29.14,4.44,1.68,3.69
2,a/0009,304,70,222,52,153,35.58,3.99,2.86,3.31
3,a/0017,83,417,411,54,175,44.58,1.00,14.46,1.00
4,a/0019,134,2257,2104,152,999,155.61,1.23,59.46,1.00


In [36]:
display_stats_table(experiment_instances["b"])

,instance,#aisles,#orders,#items,LB,UB,mean_aisle_capacity,mean_order_size,mean_items_per_aisle,mean_items_per_order
0,b/0001,185,11321,7675,1206,3968,985.13,1.32,140.59,1.00
1,b/0003,164,2070,5617,585,2150,1126.27,6.29,94.54,3.84
2,b/0005,179,6468,4959,996,2869,705.64,1.47,88.96,1.00
3,b/0007,165,1841,1621,91,833,295.44,1.27,36.46,1.00
4,b/0009,417,5581,4990,1123,3205,118.60,1.31,35.72,1.00


In [35]:
display_stats_table(experiment_instances["x"])

,instance,#aisles,#orders,#items,LB,UB,mean_aisle_capacity,mean_order_size,mean_items_per_aisle,mean_items_per_order
0,x/0001,164,1949,4608,810,2380,770.74,3.78,79.86,2.83
1,x/0003,180,11330,7399,1831,3958,1032.26,1.32,132.56,1.00
2,x/0006,168,2704,6523,1196,2979,1353.11,5.22,110.38,3.62
3,x/0007,418,8725,7715,1490,5518,162.80,1.27,52.93,1.00
4,x/0008,482,20364,17991,3937,10520,926.88,1.22,243.01,1.00


Ejecutar con algoritmo greedy

In [ ]:

for dataset in ["a", "b", "x"]:
    for instance in experiment_instances[dataset]:
        
        print(f"Processing instance: {dataset}/{instance.id}")
        !java -jar target/ChallengeSBPO2025-1.0.jar datasets/{dataset}/instance_{instance.id}.txt greedy
        print("\n")


Verificación de factibilidad y valores objetivo

In [37]:
# %%capture
checker = WaveOrderPicking()

algorithm = "greedy"

for dataset in ["a", "b", "x"]:
    for instance in experiment_instances[dataset]:
        result = checker.check_result(f"datasets/{dataset}/instance_{instance.id}.txt", f"output/{algorithm}/{dataset}/instance_{instance.id}.txt")
        print(f"Result for instance {dataset}/instance_{instance.id}.txt: {result}")

Result for instance a/instance_0001.txt: {'is_feasible': True, 'objective_value': 9.4}
Result for instance a/instance_0004.txt: {'is_feasible': True, 'objective_value': 1.375}
Result for instance a/instance_0009.txt: {'is_feasible': True, 'objective_value': 1.4266666666666667}
Result for instance a/instance_0017.txt: {'is_feasible': True, 'objective_value': 15.875}
Result for instance a/instance_0019.txt: {'is_feasible': True, 'objective_value': 202.0}
Result for instance b/instance_0001.txt: {'is_feasible': True, 'objective_value': 329.3636363636364}
Result for instance b/instance_0003.txt: {'is_feasible': True, 'objective_value': 110.57894736842105}
Result for instance b/instance_0005.txt: {'is_feasible': True, 'objective_value': 207.15384615384616}
Result for instance b/instance_0007.txt: {'is_feasible': True, 'objective_value': 52.0}
Result for instance b/instance_0009.txt: {'is_feasible': True, 'objective_value': 45.026315789473685}
Result for instance x/instance_0001.txt: {'is_fe

Mejores soluciones publicadas

In [38]:
for dataset in ["a", "b", "x"]:
    for instance in experiment_instances[dataset]:
        result = checker.check_result(f"datasets/{dataset}/instance_{instance.id}.txt", f"best_solutions/{dataset}/instance_{instance.id}.txt")
        print(f"Result for instance {dataset}/instance_{instance.id}.txt: {result}")
        if result["is_feasible"]:
            instance.best_result = result["objective_value"]

Result for instance a/instance_0001.txt: {'is_feasible': True, 'objective_value': 15.0}
Result for instance a/instance_0004.txt: {'is_feasible': True, 'objective_value': 3.5}
Result for instance a/instance_0009.txt: {'is_feasible': True, 'objective_value': 4.416666666666667}
Result for instance a/instance_0017.txt: {'is_feasible': True, 'objective_value': 36.5}
Result for instance a/instance_0019.txt: {'is_feasible': True, 'objective_value': 202.0}
Result for instance b/instance_0001.txt: {'is_feasible': True, 'objective_value': 631.5}
Result for instance b/instance_0003.txt: {'is_feasible': True, 'objective_value': 149.6}
Result for instance b/instance_0005.txt: {'is_feasible': True, 'objective_value': 368.0}
Result for instance b/instance_0007.txt: {'is_feasible': True, 'objective_value': 108.0}
Result for instance b/instance_0009.txt: {'is_feasible': True, 'objective_value': 82.5}
Result for instance x/instance_0001.txt: {'is_feasible': True, 'objective_value': 70.84848484848484}
Re

## Experimentos

In [39]:
from entities import Experiment

experiments = {}
Experiment.default_parameters = {
        "generations" : 50,
        "population_size" : 60,
        "crossover_rate" : 0.9,
        "mutation_rate" : 0.001,
        "initial_seed" : 12345,
        "iterations" : 1
    }
independent_runs = 10

### 1. Configuración de algoritmos genéticos

In [40]:
algorithms1 = ["ssGA", "gGA"]
batch1 = "algorithm_configuration"
dataset1 = "a"

#### 1.1. Codificación de soluciones: cadenas binarias vs subconjuntos

In [41]:
batch_name = os.path.join(batch1, "encoding")
encodings = ["subset", "binary"]
experiments[batch_name] = []


for instance in experiment_instances[dataset1]:
    for algorithm in algorithms1:
        for encoding in encodings:
            for run in range(independent_runs):
                exp = Experiment(batch_name, instance, algorithm, run)
                # exp.set_param("":"")
                experiments[batch_name].append(exp)


#### 1.2. Operador de cruzamiento: HUX vs unión de subconjunto de órdenes

In [42]:
batch_name = os.path.join(batch1, "crossover")

#### 1.3. Inicialización: aleatoria o warm start

In [43]:
batch_name = os.path.join(batch1, "start")

### 2. Configuración de parámetros

In [44]:
batch2 = "parameter_tuning"

### 3. Evaluación


In [45]:
batch3 = "evaluation"

## Análisis comparativo